# Fusion

Submission 2:
    - Tremor : Gradient Boosting Regression
    - Dyskinesia : Gradient Boosting Regression
    
Submission 4: 
    - Dyskinesia : Average of predictions 

In [1]:
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
import csv
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from random import random
from sklearn.metrics import mean_squared_error
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

%load_ext autoreload
%autoreload 2

from transform_data import *
from create_graphs import *
from beatPDivec.default_data.v2_auto.local.get_final_scores_accuracy import *

In [2]:
sDirOut = "/export/b19/mpgill/BeatPD_predictions_tryingout"

## Submission 4 - Average of predictions 

In [52]:
def average_fusion(lFilesPred, sDirOut, fileName, bRound, sFileLabels, sTypeLabel):
    
    # Read file labels (true labels)
        # Read file labels (true labels)
    print("sFileLabels : ", sFileLabels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
        
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
        
    iNumFiles=len(lFilesPred)
    print('Number of analyzed file inputs: '+ str(iNumFiles))
    #mPredictions=[] #np.zeros((1,iNumFiles))
    #mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    mPredictions=np.zeros((1,iNumFiles))
    lID=[] # measurement_id 
    lDicts=[] 
    vLabels = [] # true label
    vParID=[] # participant ID

    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (acoustic, w-embed)
        lDicts.append(dPred)
   
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            lID.append(k) # measurement_id 
            vLabels.append(float(np.asarray(dLabels[k]))) #true labels
            vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            # Go through the second and next predictions files
            for j in range(1, iNumFiles):
                # fPred will contain the prediction of the predictions file we're going through
                fPred=lDicts[j].get(k)
                if fPred: # if we found a prediction
                    # we add it in the array
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
            if bEnter==1:
                # if it's the first prediction file, we initialize it
                mPredictions=vPredIter.copy() # mPredictions initialization.
                bEnter=0
            else:
                # We add the new predictions as a new column if it's not the first file we're going through
                mPredictions=np.append(mPredictions,vPredIter,axis=0)

#     print('mPredictions and vID')
#     print(mPredictions.shape)
#     print(len(lID))
#     print(mPredictions)
    vAverage=np.mean(mPredictions,axis=1)
    if bRound==1:
        vPrediction=np.round(vAverage)
    else:
        vPrediction=vAverage
    
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]
          
    # Random forest training - regression
    #clf=RandomForestClassifier(n_estimators)
    #clf = clf.fit(X, Y)
    #clf.score(X_test, y_test)

    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())
    lID, vPrediction=zip(*sorted(zip(lID, vPrediction)))
    df = pd.DataFrame({'measurement_id': lID, sTypeLabel:vPrediction})
    df.to_csv(sDirOut+'submissionCisPD'+sTypeLabel+'.csv', index=False)
    
    # we will include the testing data here
    #print(mPredictions)
#     print(len(vLabels))
    return vRes1, vRes2, vAverage, vParID, vLabels

### Dyskinesia - Submission 4 - Average

In [66]:
# tsfresh predictions file on test kfolds 
#sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
# SVR predictions files on test kfolds 
#sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'

sFilePred1='/home/mpgill/BeatPD/BeatPD-CLSP-JHU/tsfresh/submit/submission4_preds/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/b19/mpgill/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl_scratch/exp/ivec_650/resiVecSVR_Fold/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred2]

# Path to labels on CIS-PD 
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dysk_test"

sTypeLabel = "dyskinesia"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))


sFileLabels :  /export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
--- MSEscore ---
Final score :  0.4830357155225596
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  0.5144468970875267
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  0.48601343286255055
Overall MSE Fusion - average :  None


In [67]:

# tsfresh predictions file on test kfolds 
#sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
# SVR predictions files on test kfolds 
#sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'

sFilePred1='/home/mpgill/BeatPD/BeatPD-CLSP-JHU/tsfresh/submit/submission4_preds/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred2]

# Path to labels on CIS-PD 
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dysk_test"

sTypeLabel = "dyskinesia"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))




sFileLabels :  /export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
--- MSEscore ---
Final score :  0.4830357155225596
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  0.4810183429568722
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  0.4700143231441346
Overall MSE Fusion - average :  None


## Submission 2 - Gradient Boosting Regression

I think the following code was used in the second submission, but I didn't take the time on making it clear and pretty since it's not in our final submission.

In [43]:
def read_files(lFilesPred, sFileLabels, sTypeLabel):
    """
    TODO 
    
    Keyword Arguments: 
    - lFilesPred: 
    - sFileLabels: 
    - sTypeLabels: 
    """
    
    # Read file labels (true labels)
    print("sFileLabels : ", sFileLabels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
    
    # Training-testing data
    iNumFiles=len(lFilesPred)
    #mPredictions=[] #np.zeros((1,iNumFiles))
    #mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    vLabels=[] #true label
    vParID=[] # participant ID
    lDicts=[] 


    # We loop over the predictions files from the tsfresh and SVR
    # lDicts is a list of measurement_id and the predictions obtained 
    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
    print('sTypeLabel : ', sTypeLabel)
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
    
    print('dLabels')
    print(dLabels)
    
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            if dLabels[k]!='NA':
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
                vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
                for j in range(1, iNumFiles):
                    fPred=lDicts[j].get(k)
                if fPred:
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
                    
            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    print('mPredictions and vLabels')
    print(mPredictions.shape)
    print(len(vLabels))
    # Random forest training - regression
    
    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())
    return mPredictions, vLabels

In [7]:
def RandomForest_fusion(lFilesPred, sFileLabels, iEstimators, sTypeLabel, nudge, rLR, iMD,iRS):

    mPredictions, vLabels = read_files(lFilesPred, sFileLabels, sTypeLabel)
    
    clf=GradientBoostingRegressor(n_estimators=iEstimators, learning_rate=rLR, max_depth=iMD,\
                                      random_state=iRS, loss='ls').fit(mPredictions, vLabels)
    vNewPred=clf.predict(mPredictions)
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]
    
    # we will include the testing data here
    return (vNewPred, vParID, vLabels, vRes1, vRes2, clf)
#print(mPredictions)

In [8]:
def randomForest_predTest(lTestPred, clf, sDirOut,sName):
    
    lDicts=[] #Will contain the two test predictions
    lmesID=[]
    iNumFiles=len(lTestPred)
    vPredIter=np.zeros((1,iNumFiles))

    for sFilePred in lTestPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
      
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            lmesID.append(k)
            #vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            for j in range(1, iNumFiles):
                fPred=lDicts[j].get(k)
            if fPred:
                vPredIter[0,j]=float(np.asarray(fPred))
            else:
                print(['Unkwnown key:' + k])
                vPredIter[0,j]=float(np.asarray(lDicts[0][k]))

            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    
    vTestPred=clf.predict(mPredictions)
    df = pd.DataFrame({'measurement_id': lmesID, sName:vTestPred})
    df.to_csv(sDirOut+'submissionCisPD'+sName+'.csv', index=False)
    return(lmesID, vTestPred)



### Dyskinesia

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_on_off.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sFilePred2='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTypeLabel='on_off'
n_estimators=300
rLR=0.1
iMD=1
iRS=15
nudge=100
sFileLabels=sFilePred1

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1,\
vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel, nudge, rLR, iMD,iRS)

print('Overall MSE Classif. 1 : ', get_final_score(vTrueLabels,np.array(vParID).astype(int), vRes1))
print('Overall MSE Classif. 2 : ', get_final_score(vTrueLabels, vParID, vRes2))
print('Overall MSE Fusion : ', get_final_score(vPredCross, vParIDCross,vTrueCross))




sTestPred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/On_off_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/'
sName='on_off'
#lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
#print(vTestPred)




### Tremor - Submission 2

In [ ]:
# tsfresh predictions file 
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_tremor.csv'

# SVR predictions file 
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold_all/objs_450_kernel_linear_c_0.02_eps_0.1.csv'

# subchallenge 
sTypeLabel='tremor'

# RandomForest hyperparameter 
n_estimators=5

# Labels
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'


lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion(lFilesPred,
                                                                           sFileLabels,
                                                                           n_estimators,
                                                                           sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.tremor.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/Tremor_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='tremor'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



### Dyskinesia - Submission 2

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_dyskinesia.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'
sTypeLabel='dyskinesia'
n_estimators=5
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold/Dyskinesia_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='z_dyskinesia'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)

